In [274]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

1. Повнозв'язані нейронні мережі.
Вирішіть завдання класифікації даних, з якими ви працювали в лабораторній № 1 за допомогою повнозв’язаної нейромережі прямого поширення (fully connected feed-forward network). Результати порівняйте з одержаними раніше. 

In [275]:
df = pd.read_csv('../Lab 1/AIDS_Classification.csv')
df.head()

,time,trt,age,wtkg,hemo,homo,drugs,karnof,oprior,z30,...,str2,strat,symptom,treat,offtrt,cd40,cd420,cd80,cd820,infected
0,948,2,48,89.8128,0,0,0,100,0,0,...,0,1,0,1,0,422,477,566,324,0
1,1002,3,61,49.4424,0,0,0,90,0,1,...,1,3,0,1,0,162,218,392,564,1
2,961,3,45,88.4520,0,1,1,90,0,1,...,1,3,0,1,1,326,274,2063,1893,0
3,1166,3,47,85.2768,0,1,0,100,0,1,...,1,3,0,1,0,287,394,1590,966,0
4,1090,0,43,66.6792,0,1,0,100,0,1,...,1,3,0,0,0,504,353,870,782,0


In [276]:
X = df.drop(columns=['infected'])
y = df['infected']
X = np.array(X)
y = np.array(y)
object = StandardScaler()
X = object.fit_transform(X)

In [277]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.5, random_state = 40)

# Об'єкт Тензор

In [278]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.14.0


In [279]:
a = tf.constant(1)
print(a)
a = tf.constant(1, shape=(1, 1))
print(a)
b = tf.constant([[1, 2, 3]])
print(b)
b1 = b.numpy()
print(b1)

tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor([[1]], shape=(1, 1), dtype=int32)
tf.Tensor([[1 2 3]], shape=(1, 3), dtype=int32)
[[1 2 3]]


In [280]:
a1 = tf.cast(a, dtype=tf.float32)
print(a1)

tf.Tensor([[1.]], shape=(1, 1), dtype=float32)


In [281]:
v = tf.Variable([1, 2, 3])
print(v)
v.assign([-1, -2, -3])
print(v)

<tf.Variable 'Variable:0' shape=(3,) dtype=int32, numpy=array([1, 2, 3])>
<tf.Variable 'Variable:0' shape=(3,) dtype=int32, numpy=array([-1, -2, -3])>


In [282]:
a = tf.eye(3, 3)
print(a)
a1 = np.eye(3, 3)
print(a1)

tf.Tensor(
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]], shape=(3, 3), dtype=float32)
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [283]:
a = tf.zeros((3, 3))
print(a)
a1 = np.zeros((3, 3))
print(a1)
b = tf.zeros_like(a)
print(b)
b1 = tf.identity(b)
print(b1)

tf.Tensor(
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]], shape=(3, 3), dtype=float32)
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
tf.Tensor(
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]], shape=(3, 3), dtype=float32)
tf.Tensor(
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]], shape=(3, 3), dtype=float32)


In [284]:
a = np.array([1, 2, 3])
b = np.array([-1, -2, -3])
print(a * b)
print(np.tensordot(a, b, axes=0))
print(np.tensordot(a, b, axes=1))
print(np.sum(a * b))

[-1 -4 -9]
[[-1 -2 -3]
 [-2 -4 -6]
 [-3 -6 -9]]
-14
-14


In [285]:
a = tf.Variable([1, 2, 3])
b = tf.Variable([-1, -2, -3])
print(a * b)
print(tf.tensordot(a, b, axes=0))
print(tf.tensordot(a, b, axes=1))
print(np.sum(a * b))

tf.Tensor([-1 -4 -9], shape=(3,), dtype=int32)
tf.Tensor(
[[-1 -2 -3]
 [-2 -4 -6]
 [-3 -6 -9]], shape=(3, 3), dtype=int32)
tf.Tensor(-14, shape=(), dtype=int32)
-14


# Автоматичне диференціювання

In [286]:
y = lambda x: x ** 2
h = 0.001
x_0 = -2
diff_y = (y(x_0 + h) - y(x_0 - h)) / 2 * h
print(diff_y)

-3.999999999999559e-06


In [287]:
x_0 = tf.Variable(-2.0)
print(x_0)
with tf.GradientTape() as tape:
    y = x_0 ** 2
print(y)
diff_y = tape.gradient(y, x_0)
diff_y = diff_y.numpy()
print(diff_y)

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=-2.0>
tf.Tensor(4.0, shape=(), dtype=float32)
-4.0


In [288]:
w = tf.Variable(tf.random.normal((3, 2)), name='w')
b = tf.Variable(tf.zeros(2, dtype=tf.float32), name='b')
x = [[1., 2., 3.]]

with tf.GradientTape(persistent=True) as tape:
  y = x @ w + b
  l = tf.reduce_mean(y**2)

dl_dw, dl_db = tape.gradient(l, [w, b])
print(dl_dw.numpy(), dl_db.numpy(), sep='\n')

del tape

[[-1.3917973  4.794115 ]
 [-2.7835946  9.58823  ]
 [-4.175392  14.382345 ]]
[-1.3917973  4.794115 ]


# Градієнтні алгоритми оптимізації

In [289]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import Accuracy

In [290]:
model = Sequential([
    Dense(100, activation="relu", input_shape=(X_train.shape[1],)),  
    Dense(100, activation="relu"),  
    Dense(1, activation="sigmoid")
])

In [291]:
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss=BinaryCrossentropy(),
    metrics=["accuracy"]
)

In [292]:
model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=35,
    batch_size=32
)

Epoch 1/35
34/34 [==============================] - 2s 13ms/step - loss: 0.5386 - accuracy: 0.7259 - val_loss: 0.4641 - val_accuracy: 0.7813
Epoch 2/35
34/34 [==============================] - 0s 5ms/step - loss: 0.4149 - accuracy: 0.8026 - val_loss: 0.4018 - val_accuracy: 0.8112
Epoch 3/35
34/34 [==============================] - 0s 5ms/step - loss: 0.3652 - accuracy: 0.8316 - val_loss: 0.3703 - val_accuracy: 0.8346
Epoch 4/35
34/34 [==============================] - 0s 6ms/step - loss: 0.3368 - accuracy: 0.8466 - val_loss: 0.3538 - val_accuracy: 0.8439
Epoch 5/35
34/34 [==============================] - 0s 5ms/step - loss: 0.3123 - accuracy: 0.8559 - val_loss: 0.3399 - val_accuracy: 0.8486
Epoch 6/35
34/34 [==============================] - 0s 5ms/step - loss: 0.2947 - accuracy: 0.8709 - val_loss: 0.3288 - val_accuracy: 0.8607
Epoch 7/35
34/34 [==============================] - 0s 5ms/step - loss: 0.2775 - accuracy: 0.8803 - val_loss: 0.3243 - val_accuracy: 0.8626
Epoch 8/35
34/34 [=

In [293]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

34/34 [==============================] - 0s 1ms/step - loss: 0.3864 - accuracy: 0.8813
Test Loss: 0.3864
Test Accuracy: 0.8813


2. Згорткові нейронні мережі

Вирішіть завдання класифікації зображень за допомогою згорткової (convolutional) нейромережі двома способами:

а. навчить мережу з нуля (from scratch)

б. застосуйте перенесення навчання (transfer learning from pre-trained weights)

Порівняйте результати (якщо в обраному датасеті класів забагато, достатньо залишити 3-5).

In [294]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

In [297]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
selected_classes = [0, 1, 3]
mask_train = np.isin(y_train, selected_classes)
mask_test = np.isin(y_test, selected_classes)
x_train, y_train = x_train[mask_train.flatten()], y_train[mask_train.flatten()]
x_test, y_test = x_test[mask_test.flatten()], y_test[mask_test.flatten()]
class_mapping = {0: 0, 1: 1, 3: 2}  # airplane -> 0, car -> 1, cat -> 2
y_train = np.array([class_mapping[label[0]] for label in y_train])
y_test = np.array([class_mapping[label[0]] for label in y_test])
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train = to_categorical(y_train, num_classes=3)
y_test = to_categorical(y_test, num_classes=3)

In [298]:
model_from_scratch = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')
])

In [299]:
model_from_scratch.compile(optimizer='adam',
                           loss='categorical_crossentropy',
                           metrics=['accuracy'])

In [300]:
history_scratch = model_from_scratch.fit(
    x_train, y_train,
    validation_data=(x_test, y_test),
    epochs=10,
    batch_size=32
)

Epoch 1/10
469/469 [==============================] - 15s 26ms/step - loss: 0.5833 - accuracy: 0.7636 - val_loss: 0.4109 - val_accuracy: 0.8407
Epoch 2/10
469/469 [==============================] - 12s 26ms/step - loss: 0.3897 - accuracy: 0.8546 - val_loss: 0.3255 - val_accuracy: 0.8750
Epoch 3/10
469/469 [==============================] - 12s 26ms/step - loss: 0.3328 - accuracy: 0.8766 - val_loss: 0.2950 - val_accuracy: 0.8907
Epoch 4/10
469/469 [==============================] - 12s 25ms/step - loss: 0.2921 - accuracy: 0.8916 - val_loss: 0.2691 - val_accuracy: 0.9023
Epoch 5/10
469/469 [==============================] - 11s 24ms/step - loss: 0.2587 - accuracy: 0.9028 - val_loss: 0.2685 - val_accuracy: 0.9040
Epoch 6/10
469/469 [==============================] - 12s 26ms/step - loss: 0.2328 - accuracy: 0.9160 - val_loss: 0.2615 - val_accuracy: 0.9067
Epoch 7/10
469/469 [==============================] - 13s 27ms/step - loss: 0.2023 - accuracy: 0.9271 - val_loss: 0.2546 - val_accuracy:

# Перенесення навчання (VGG16)

In [301]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

58889256/58889256 [==============================] - 8s 0us/step


In [305]:
for layer in base_model.layers:
    layer.trainable = False

model_transfer = Sequential([
    base_model,
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')
])
model_transfer.compile(optimizer='adam',
                       loss='categorical_crossentropy',
                       metrics=['accuracy'])
history_transfer = model_transfer.fit(
    x_train, y_train,
    validation_data=(x_test, y_test),
    epochs=5,
    batch_size=32
)

Epoch 1/5
469/469 [==============================] - 78s 163ms/step - loss: 0.5546 - accuracy: 0.7737 - val_loss: 0.4371 - val_accuracy: 0.8257
Epoch 2/5
469/469 [==============================] - 76s 163ms/step - loss: 0.4366 - accuracy: 0.8313 - val_loss: 0.4043 - val_accuracy: 0.8413
Epoch 3/5
469/469 [==============================] - 77s 164ms/step - loss: 0.4069 - accuracy: 0.8423 - val_loss: 0.3837 - val_accuracy: 0.8473
Epoch 4/5
469/469 [==============================] - 76s 162ms/step - loss: 0.3839 - accuracy: 0.8515 - val_loss: 0.3741 - val_accuracy: 0.8510
Epoch 5/5
469/469 [==============================] - 76s 163ms/step - loss: 0.3712 - accuracy: 0.8585 - val_loss: 0.3713 - val_accuracy: 0.8483


In [306]:
loss_scratch, acc_scratch = model_from_scratch.evaluate(x_test, y_test, verbose=0)
loss_transfer, acc_transfer = model_transfer.evaluate(x_test, y_test, verbose=0)

print(f"Model from Scratch: Accuracy = {acc_scratch:.4f}, Loss = {loss_scratch:.4f}")
print(f"Transfer Learning: Accuracy = {acc_transfer:.4f}, Loss = {loss_transfer:.4f}")

Model from Scratch: Accuracy = 0.9137, Loss = 0.2578
Transfer Learning: Accuracy = 0.8483, Loss = 0.3713


3. Рекурентні нейронні мережі

Вирішіть задачу класифікації текстів (з якими ви працювали в лабораторній № 2) за допомогою рекурентної нейромережі двома способами:

а) навчить мережу і embedding шар з нуля (from scratch)

б) використовуючи pretrained word embeddings

 Результати порівняйте між собою і з одержаними раніш. 

In [307]:
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

In [308]:
df = pd.read_csv('../Lab 2/Leetcode_Questions.csv')
df.head()

,Question_No,Question,Acceptance,isPremium,Difficulty,Question_Link,Solution
0,1,Two Sum,53.80%,False,Easy,https://leetcode.com/problems/two-sum/description,https://leetcode.com/problems/two-sum/solutions
1,2,Add Two Numbers,44.10%,False,Medium,https://leetcode.com/problems/add-two-numbers/...,https://leetcode.com/problems/add-two-numbers/...
2,3,Longest Substring Without Repeating Characters,35.40%,False,Medium,https://leetcode.com/problems/longest-substrin...,https://leetcode.com/problems/longest-substrin...
3,4,Median of Two Sorted Arrays,41.50%,False,Hard,https://leetcode.com/problems/median-of-two-so...,https://leetcode.com/problems/median-of-two-so...
4,5,Longest Palindromic Substring,34.40%,False,Medium,https://leetcode.com/problems/longest-palindro...,https://leetcode.com/problems/longest-palindro...


In [309]:
label_encoder = LabelEncoder()
df["Difficulty"] = label_encoder.fit_transform(df["Difficulty"])  # Easy=0, Medium=1, Hard=2
y = to_categorical(df["Difficulty"])  # One-hot encoding

In [310]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df["Question"])
vocab_size = len(tokenizer.word_index) + 1

In [311]:
X = tokenizer.texts_to_sequences(df["Question"])
max_len = max(len(seq) for seq in X)
X = pad_sequences(X, maxlen=max_len, padding="post")

In [312]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [313]:
model_from_scratch = Sequential([
    Embedding(input_dim=vocab_size, output_dim=50, input_length=max_len),
    Bidirectional(LSTM(64, return_sequences=False)),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax')
])

In [314]:
model_from_scratch.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [315]:
model_from_scratch.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=10,
    batch_size=16
)

Epoch 1/10
146/146 [==============================] - 8s 19ms/step - loss: 1.0321 - accuracy: 0.5210 - val_loss: 1.0123 - val_accuracy: 0.5352
Epoch 2/10
146/146 [==============================] - 2s 14ms/step - loss: 0.9990 - accuracy: 0.5232 - val_loss: 0.9973 - val_accuracy: 0.5283
Epoch 3/10
146/146 [==============================] - 2s 13ms/step - loss: 0.8542 - accuracy: 0.6206 - val_loss: 1.0341 - val_accuracy: 0.4786
Epoch 4/10
146/146 [==============================] - 2s 11ms/step - loss: 0.6416 - accuracy: 0.7408 - val_loss: 1.1344 - val_accuracy: 0.5043
Epoch 5/10
146/146 [==============================] - 2s 14ms/step - loss: 0.5183 - accuracy: 0.7876 - val_loss: 1.2340 - val_accuracy: 0.4957
Epoch 6/10
146/146 [==============================] - 2s 14ms/step - loss: 0.4370 - accuracy: 0.8249 - val_loss: 1.3946 - val_accuracy: 0.4768
Epoch 7/10
146/146 [==============================] - 2s 12ms/step - loss: 0.3888 - accuracy: 0.8455 - val_loss: 1.4507 - val_accuracy: 0.4889